In [1]:
import scipy.signal
import sys
import torch
import torch.nn as nn
import numpy as np

In [2]:
from typing import Dict, List, Optional, Tuple
import gym
from PIL import Image
# from pyvirtualdisplay import Display
# Display().start()
from datetime import datetime

In [3]:
import math
import random
from copy import deepcopy
import torch
from torch.optim import Adam
from torch.optim import RMSprop
import gym
import time
from collections import namedtuple, deque
import neptune.new as neptune

In [4]:
import robosuite as suite
from robosuite.controllers import load_controller_config
from robosuite.controllers.controller_factory import reset_controllers
from robosuite.utils import observables
from robosuite.utils.input_utils import *
from robosuite.robots import Bimanual

In [5]:
nep_log = neptune.init(
    project="xhnfirst/DDPG-mian-test1",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1NTg5MDI2OS01MTVmLTQ2YjUtODA1Yy02ZWQyNDgxZDcwN2UifQ==",
)

https://app.neptune.ai/xhnfirst/DDPG-mian-test1/e/DDPGMIAN-159
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [6]:
import math
import sys
from time import strftime
from time import gmtime

class view_bar:
    def __init__(self, message, num, total, value_name, value_data, start_time_rec, r_true, reward_ctrl, reward_dist, total_main):
        self.message = message
        self.num = num
        self.total = total
        self.message2 = "True time of r = "
        self.r_true = r_true
        self.message3 = "time cost = "
        rate = self.num  / self.total
        rate_num = int(rate * 40)
        rate_nums = math.ceil(rate * 100)
        time_rec = datetime.now()
        execution_time = time_rec - start_time_rec
        self.message4 = "{reward_ctrl: "
        self.reward_ctrl = reward_ctrl
        self.message5 = "reward_dist: "
        self.reward_dist = reward_dist
        self.message6 = "Total Reward Main: "
        self.total_main = total_main
        self.message8 = " }"

        r = '\r%s:[%s%s]%d%%\t%d/%d\t%s%f\t%s%s\t%s%s\t\t\t\t%s%.4f\t%s%.4f\t%s%.4f\t%s' % (self.message, "=" * rate_num,
                                        " " * (40 - rate_num), rate_nums, self.num , self.total, value_name, value_data, 
                                        self.message3, execution_time, self.message2, str(self.r_true), str(self.message4), self.reward_ctrl,
                                        str(self.message5), self.reward_dist, str(self.message6), self.total_main, 
                                         str(self.message8)
                                        )


        sys.stdout.write(r)
        sys.stdout.flush()




In [7]:
options = {
    'env_name': 'EElab_test2',
    "robots": "UR5e"
}
controller_name = "JOINT_VELOCITY"
options["controller_configs"] = suite.load_controller_config(default_controller=controller_name)

action_dim = 6
num_test_steps = 6
test_value = 1
steps_per_action = 75
steps_per_rest = 75

env = suite.make(
    **options,
    has_renderer=True,
    has_offscreen_renderer=False,
    ignore_done=True,
    use_camera_obs=False,
    horizon=(steps_per_action + steps_per_rest) * num_test_steps,
    control_freq=20,
)


test_env = suite.make(
    **options,
    has_renderer=True,
    has_offscreen_renderer=False,
    ignore_done=True,
    use_camera_obs=False,
    horizon=(steps_per_action + steps_per_rest) * num_test_steps,
    control_freq=20,
)


device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device = ', device)

Creating window glfw
Creating window glfw
device =  cuda


In [8]:
def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)


class MLPActor(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation, act_limit):
        super().__init__()
        pi_sizes = [obs_dim] + list(hidden_sizes) + [act_dim]
        self.pi = mlp(pi_sizes, activation, nn.Tanh)
        self.act_limit = act_limit

    def forward(self, obs):
        # Return output from network scaled to action space limits.
        return self.act_limit * self.pi(obs)

class MLPQFunction(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.q = mlp([obs_dim + act_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs, act):
        q = self.q(torch.cat([obs, act], dim=-1))
        return torch.squeeze(q, -1) # Critical to ensure q has right shape.

class MLPActorCritic(nn.Module):

    def __init__(self, hidden_sizes=(256,256),
                 activation=nn.ReLU, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        super().__init__()

        obs_dim = 60
        act_dim = 7
        act_limit = 1

        # build policy and value functions
        self.pi = MLPActor(obs_dim, act_dim, hidden_sizes, activation, act_limit).to(device)
        self.q = MLPQFunction(obs_dim, act_dim, hidden_sizes, activation).to(device)

    def act(self, obs):
        with torch.no_grad():
            return self.pi(obs)

In [9]:
Transition = namedtuple('Transition',
                        ('obs', 'act', 'rew', 'next_obs', 'done'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [10]:

params = {
    "batch_size": 128,
    "dropout": 0.2,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "hid": 64,
    "l": 3,
    "seed": 0,
    "steps_per_epoch": 3000,
    "epochs": 100,
    "replay_size": int(1e6),
    "gamma": 0.99,
    "polyak": 0.995,
    "pi_lr": 1e-4,
    "q_lr": 1e-4,
    "batch_size": 500,
    "start_steps": 10000, 
    "update_after": 1000,
    "update_every": 50,
    "act_noise": 0.01,
    "num_test_episodes": 10,
    "max_ep_len": 1000
}


ac_kwargs=dict(hidden_sizes=[params["hid"]]*params["l"])

In [11]:
nep_log["parameters"] = params

torch.manual_seed(params["seed"])
np.random.seed(params["seed"])

obs_dim = 60
print('obs_dim = ', obs_dim)
act_dim = 7
print('act_dim = ', act_dim)
# Action limit for clamping: critically, assumes all dimensions share the same bound!
act_limit = 1
print('act_limit = ', act_limit)
# Create actor-critic module and target networks
ac = MLPActorCritic(**ac_kwargs)
ac_targ = deepcopy(ac)

# Freeze target networks with respect to optimizers (only update via polyak averaging)
for p in ac_targ.parameters():
    p.requires_grad = False

memory = ReplayMemory(params["replay_size"])

obs_dim =  60
act_dim =  7
act_limit =  1


In [12]:
# Set up function for computing DDPG Q-loss
def compute_loss_q(data):

    o = torch.cat(data.obs).float()
    a = torch.cat(data.act).float()
    r = torch.cat(data.rew).float()
    o2 =torch.cat(data.next_obs).float()
    d = torch.cat(data.done).float()

    q = ac.q(o,a)


    # Bellman backup for Q function
    with torch.no_grad():
        q_pi_targ = ac_targ.q(o2, ac_targ.pi(o2))
        backup = r + params["gamma"] * (1 - d) * q_pi_targ

    # MSE loss against Bellman backup
    loss_q = ((q - backup)**2).mean()

    return loss_q

# Set up function for computing DDPG pi loss
def compute_loss_pi(data):

    o = torch.cat(data.obs).float()

    q_pi = ac.q(o, ac.pi(o))

    return -q_pi.mean()


In [13]:
pi_optimizer = RMSprop(ac.pi.parameters(), lr=params["pi_lr"])
q_optimizer = RMSprop(ac.q.parameters(), lr=params["q_lr"])

def update(data):
    # First run one gradient descent step for Q.


    q_optimizer.zero_grad()
    loss_q = compute_loss_q(data)

    loss_q.backward()

    q_optimizer.step()


    # Freeze Q-network so you don't waste computational effort 
    # computing gradients for it during the policy learning step.
    for p in ac.q.parameters():
        p.requires_grad = False

    # Next run one gradient descent step for pi.
    pi_optimizer.zero_grad()
    loss_pi = compute_loss_pi(data)
    loss_pi.backward()
    pi_optimizer.step()

    # Unfreeze Q-network so you can optimize it at next DDPG step.
    for p in ac.q.parameters():
        p.requires_grad = True

    nep_log


    # Finally, update target networks by polyak averaging.
    with torch.no_grad():
        for p, p_targ in zip(ac.parameters(), ac_targ.parameters()):
            # NB: We use an in-place operations "mul_", "add_" to update target
            # params, as opposed to "mul" and "add", which would make new tensors.
            p_targ.data.mul_(params["polyak"])
            p_targ.data.add_((1 - params["polyak"]) * p.data)


In [14]:



def get_action(o, noise_scale):
    a = ac.act(torch.tensor(o, dtype=torch.float32, device=device))
    a += noise_scale * torch.randn(act_dim).to(device)
    return torch.clip(a, -act_limit, act_limit)

def test_agent(epoch):
    
    # im = Image.fromarray(screen)
    # images = [im]
    for j in range(params["num_test_episodes"]):
        obs, d, test_ep_ret, test_ep_len = test_env.reset(), False, 0, 0
        o = list(obs['robot0_proprio-state']) + list(obs['object-state'])
        if j == 9:
            test_env.render()
            time.sleep(5)
        while not(d or (test_ep_len == params["max_ep_len"])):
            # Take deterministic actions at test time (noise_scale=0)
            a_cpu = get_action(o, 0).cpu().data.numpy()
            # a = get_action(o, 0)
            if j == 9:
                test_env.render()
                # images.append(Image.fromarray(screen))
                time.sleep(0.02)
            obs, r, d, _ = test_env.step(a_cpu)
            o = list(obs['robot0_proprio-state']) + list(obs['object-state'])
            # if np.abs(reward_dict["reward_dist"]) < 0.1 and np.abs(reward_dict["reward_ctrl"]) < 1.5:
            #     r += 2
            #     if np.abs(reward_dict["reward_dist"]) < 0.05 and np.abs(reward_dict["reward_ctrl"]) < 1:
            #         r += 4
            #         if np.abs(reward_dict["reward_dist"]) < 0.03 and np.abs(reward_dict["reward_ctrl"]) < 0.7:
            #             r += 8
            #             if np.abs(reward_dict["reward_dist"]) < 0.02 and np.abs(reward_dict["reward_ctrl"]) < 0.4:
            #                 r += 16
            #                 if np.abs(reward_dict["reward_dist"]) < 0.01 and np.abs(reward_dict["reward_ctrl"]) < 0.2:
            #                     r += 32
            #                     if np.abs(reward_dict["reward_dist"]) < 0.005 and np.abs(reward_dict["reward_ctrl"]) < 0.05:
            #                         r += 64
            #                         if np.abs(reward_dict["reward_dist"]) < 0.001 and np.abs(reward_dict["reward_ctrl"]) < 0.01:
            #                             r += 128
            #                             if np.abs(reward_dict["reward_dist"]) < 0.0001 and np.abs(reward_dict["reward_ctrl"]) < 0.001:
            #                                 r += 256
            # else:
            #     r += -0.1
            test_ep_ret += r
            test_ep_len += 1
        nep_log["test/reward"].log(test_ep_ret/test_ep_len)
    
    now = datetime.now()

    current_time = str(now.isoformat())



In [15]:
# obs = {
#     'robot0_joint_pos_cos': None,
#     'robot0_joint_pos_sin': None,
#     'robot0_joint_vel': None,
#     'robot0_eef_pos': None,
#     'robot0_eef_quat': None,
#     'robot0_gripper_qpos': None,
#     'robot0_gripper_qvel': None,
#     'cubeA_pos': None,
#     'cubeA_quat': None,
#     'cubeB_pos': None,
#     'cubeB_quat': None,
#     'gripper_to_cubeA': None,
#     'gripper_to_cubeB': None,
#     'cubeA_to_cubeB': None,
# }

obs, ep_ret, ep_len = env.reset(), 0, 0

o = list(obs['robot0_proprio-state']) + list(obs['object-state'])

env.viewer.set_camera(camera_id=0)


# Define neutral value
neutral = np.zeros(7)
# print("neutral=", neutral)
# Keep track of done variable to know when to break loop

# Prepare for interaction with environment
total_steps = params["steps_per_epoch"] * params["epochs"]
start_time = time.time()

o = torch.tensor([o], device=device)

start_time_rec = datetime.now()
r_true = 0
total_main = 0
ep_ret_main = 0
reward_dict={}

Creating window glfw


In [16]:
# Main loop: collect experience in env and update/log each epoch
low, high = env.action_spec

for t in range(total_steps):
    
    # Until start_steps have elapsed, randomly sample actions
    # from a uniform distribution for better exploration. Afterwards, 
    # use the learned policy (with some noise, via act_noise). 
    if t > params["start_steps"]:
        a = get_action(o, params["act_noise"])
        # a_cpu = a
        a_cpu = a.cpu().data.numpy()
        
    else:
        a = np.random.uniform(low, high)
        a_cpu = a
        a = torch.tensor([a], device=device)
    

    # Step the env
    obs2, r, d, _ = env.step(a_cpu)
    o2 = list(obs2['robot0_proprio-state']) + list(obs2['object-state'])

    # nep_log["testreward/dist"].log(reward_dict["reward_dist"])
    # nep_log["testreward/ctrl"].log(reward_dict["reward_ctrl"])
        
    # r = np.log(np.abs(reward_dict["reward_ctrl"]))*(-0.01) + np.log(np.abs(reward_dict["reward_dist"]))*(-0.1)

    # if np.abs(reward_dict["reward_dist"]) < 0.1 and np.abs(reward_dict["reward_ctrl"]) < 1.5:
    #     r += 2
    #     if np.abs(reward_dict["reward_dist"]) < 0.05 and np.abs(reward_dict["reward_ctrl"]) < 1:
    #         r += 4
    #         if np.abs(reward_dict["reward_dist"]) < 0.03 and np.abs(reward_dict["reward_ctrl"]) < 0.7:
    #             r += 8
    #             if np.abs(reward_dict["reward_dist"]) < 0.02 and np.abs(reward_dict["reward_ctrl"]) < 0.4:
    #                 r += 16
    #                 if np.abs(reward_dict["reward_dist"]) < 0.01 and np.abs(reward_dict["reward_ctrl"]) < 0.2:
    #                     r += 32
    #                     if np.abs(reward_dict["reward_dist"]) < 0.005 and np.abs(reward_dict["reward_ctrl"]) < 0.05:
    #                         r += 64
    #                         if np.abs(reward_dict["reward_dist"]) < 0.001 and np.abs(reward_dict["reward_ctrl"]) < 0.01:
    #                             r += 128
    #                             if np.abs(reward_dict["reward_dist"]) < 0.0001 and np.abs(reward_dict["reward_ctrl"]) < 0.001:
    #                                 r += 256
    #     r_true += 1
    # else:
    #     r += -0.1


    
    ep_len += 1
    total_main += r


    # Ignore the "done" signal if it comes from hitting the time
    # horizon (that is, when it's an artificial terminal signal
    # that isn't based on the agent's state)
    d = False if ep_len==params["max_ep_len"] else d

    # a = torch.tensor([a], device=device)
    o2 = torch.tensor([o2], device=device)
    r = torch.tensor([r], device=device)
    d = torch.tensor([d], device=device)

    # Store experience to replay buffer
    memory.push(o, a, r, o2, d)
    # Super critical, easy to overlook step: make sure to update 
    # most recent observation!
    o=o2
    ep_ret += r
    
    
    # End of trajectory handling
    if d or (ep_len == params["max_ep_len"]):
        sum_ep_ret = ep_ret
        obs, ep_ret, ep_len = env.reset(), 0, 0
        o = list(obs['robot0_proprio-state']) + list(obs['object-state'])
        o = torch.tensor([o], device=device)


    # Update handling
    if t >= params["update_after"] and t % params["update_every"] == 0:
        # view_bar("steps: ", t, total_steps, 'main of rewards = ', float(ep_ret_main), start_time_rec, 
        #         r_true, reward_dict["reward_ctrl"], reward_dict["reward_dist"], total_main/t)
        view_bar("steps: ", t, total_steps, 'main of rewards = ', float(ep_ret_main), start_time_rec, 
                r_true, 0, 0, total_main/t)
        
        nep_log["train/reward"].log(ep_ret_main)
        for i in range(params["update_every"]):

            transitions = memory.sample(params["batch_size"])
            # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
            # detailed explanation). This converts batch-array of Transitions
            # to Transition of batch-arrays.
            batch = Transition(*zip(*transitions))

            # print('batch = ', str(batch))
            update(data=batch)

    # End of epoch handling
    if (t+1) % params["steps_per_epoch"] == 0:
        epoch = (t+1) // params["steps_per_epoch"]
        ep_ret_main = sum_ep_ret/epoch
        # Test the performance of the deterministic version of the agent.
        test_agent(epoch)



/tmp/ipykernel_7031/3929466715.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180588308/work/torch/csrc/utils/tensor_new.cpp:201.)
  a = torch.tensor([a], device=device)


Creating window glfw
steps: :[                                        ]1%	1000/300000	main of rewards = 0.000000	time cost = 0:00:19.051366	True time of r = 0				{reward_ctrl: 0.0000	reward_dist: 0.0000	Total Reward Main: 0.0000	 }

RuntimeError: mat1 and mat2 shapes cannot be multiplied (500x54 and 67x64)